### Initiate h2o cluster to run Kmeans

In [1]:
library(h2o)
h2o.init(nthreads = -1, max_mem_size = '90G')


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following object is masked from ‘package:RevoScaleR’:

    colnames

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc



 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 hours 23 minutes 
    H2O cluster version:        3.16.0.2 
    H2O cluster version age:    2 months and 27 days  
    H2O cluster name:           H2O_started_from_R_capsops_xoe811 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   67.88 GB 
    H2O cluster total cores:    16 
    H2O cluster allowed cores:  16 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.4.1 (2017-06-30) 



#### Data Munging

In [2]:
predictors <- h2o.importFile(path = "https://s3-us-west-2.amazonaws.com/data516project/data/allDataCleaned.csv"
                           , destination_frame = "predictors")

predictors$C1 <- NULL

  |======================================================================| 100%


In [3]:
## Get feature lists
lasso_features1 <- read.csv('lassoFeatures1_categorized.csv')
lasso_features2 <- read.csv('lassoFeatures2_categorized.csv')

In [4]:
## Subset lasso_features1 for clustering and split training and validation frames
full <- predictors[, c(as.character(lasso_features1$Variable))]

In [5]:
dim(full)

[1] 4732698      75

#### PCA transformation on the group of variables

In [6]:
table(lasso_features1$Category)


                Auto score     Credit and Delinquency 
                         1                         18 
              Credit Score                    Finance 
                         1                          9 
                    Health                    Housing 
                         1                          8 
                    Income                Life Status 
                         1                          2 
    Neighborhood - Housing      Neighborhood - Income 
                         3                          4 
Neighborhood - Life Status  Neighborhood - Occupation 
                         1                          4 
   Neighborhood- Education                      Trade 
                         1                         21 

In [7]:
pca_features1 <- lasso_features1[lasso_features1$Category == "Credit and Delinquency", ]$Variable
pca_features2 <- lasso_features1[lasso_features1$Category == "Finance", ]$Variable
pca_features3 <- lasso_features1[lasso_features1$Category == "Housing", ]$Variable
pca_features4 <- lasso_features1[grep("Neighborhood", lasso_features1$Category), ]$Variable
pca_features5 <- lasso_features1[lasso_features1$Category == "Trade", ]$Variable

In [8]:
pca_group_variables <- function(features, k){
    training_frame = predictors[, c(as.character(features))]
    pca_fit = h2o.prcomp(training_frame, transform = "STANDARDIZE", pca_method = "GramSVD", seed = 1234, k = k,
                         impute_missing = TRUE)
    return(pca_fit)
}

#### Train PCA for each features group

In [9]:
# Credit and Delinquency
pca_fit1 <- pca_group_variables(pca_features1, 18)

# Finance
pca_fit2 <- pca_group_variables(pca_features2, 9)

# Housing
pca_fit3 <- pca_group_variables(pca_features3, 8)

# Neighborhood
pca_fit4 <- pca_group_variables(pca_features4, 13)

# Trade
pca_fit5 <- pca_group_variables(pca_features5, 21)


  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [10]:
pca_fit1
# 10 PC to achieve 80% variance explaination

Model Details:

H2ODimReductionModel: pca
Model ID:  PCA_model_R_1519665318376_34 
Importance of components: 
                            pc1      pc2      pc3      pc4      pc5      pc6
Standard deviation     1.703557 1.321111 1.292487 1.214450 1.030396 0.957053
Proportion of Variance 0.188883 0.113595 0.108726 0.095993 0.069102 0.059614
Cumulative Proportion  0.188883 0.302478 0.411203 0.507196 0.576298 0.635912
                            pc7      pc8      pc9     pc10     pc11     pc12
Standard deviation     0.943650 0.884162 0.852380 0.791477 0.733113 0.676878
Proportion of Variance 0.057956 0.050880 0.047288 0.040771 0.034980 0.029820
Cumulative Proportion  0.693869 0.744748 0.792036 0.832807 0.867787 0.897607
                           pc13     pc14     pc15     pc16     pc17     pc18
Standard deviation     0.618593 0.589542 0.569201 0.454090 0.422591 0.366382
Proportion of Variance 0.024905 0.022621 0.021087 0.013420 0.011623 0.008737
Cumulative Proportion  0.922512 0.945133 0.

In [11]:
pca_fit2
# 4 PC to achieve 80% variance explaination

Model Details:

H2ODimReductionModel: pca
Model ID:  PCA_model_R_1519665318376_35 
Importance of components: 
                            pc1      pc2      pc3      pc4      pc5      pc6
Standard deviation     1.664275 1.139534 0.930131 0.834485 0.677704 0.568163
Proportion of Variance 0.400599 0.187808 0.125126 0.100716 0.066426 0.046688
Cumulative Proportion  0.400599 0.588407 0.713533 0.814248 0.880675 0.927363
                            pc7      pc8      pc9
Standard deviation     0.525510 0.356451 0.314658
Proportion of Variance 0.039941 0.018376 0.014320
Cumulative Proportion  0.967304 0.985680 1.000000


H2ODimReductionMetrics: pca

No model metrics available for PCA



In [12]:
pca_fit3
# 6 PC to achieve 80% variance explaination

Model Details:

H2ODimReductionModel: pca
Model ID:  PCA_model_R_1519665318376_36 
Importance of components: 
                            pc1      pc2      pc3      pc4      pc5      pc6
Standard deviation     1.176689 1.146177 0.952183 0.842264 0.788457 0.757577
Proportion of Variance 0.218369 0.207191 0.142991 0.111883 0.098044 0.090515
Cumulative Proportion  0.218369 0.425559 0.568550 0.680433 0.778477 0.868992
                            pc7      pc8
Standard deviation     0.692273 0.592814
Proportion of Variance 0.075583 0.055425
Cumulative Proportion  0.944575 1.000000


H2ODimReductionMetrics: pca

No model metrics available for PCA



In [13]:
pca_fit4
# 7 PC to achieve 80% variance explaination

Model Details:

H2ODimReductionModel: pca
Model ID:  PCA_model_R_1519665318376_37 
Importance of components: 
                            pc1      pc2      pc3      pc4      pc5      pc6
Standard deviation     1.839093 1.378522 1.244483 1.134288 0.963519 0.854839
Proportion of Variance 0.262912 0.147717 0.120387 0.100011 0.072164 0.056803
Cumulative Proportion  0.262912 0.410629 0.531017 0.631028 0.703192 0.759996
                            pc7      pc8      pc9     pc10     pc11     pc12
Standard deviation     0.823014 0.810653 0.721892 0.650289 0.603279 0.507489
Proportion of Variance 0.052652 0.051083 0.040509 0.032871 0.028290 0.020020
Cumulative Proportion  0.812648 0.863730 0.904239 0.937110 0.965401 0.985420
                           pc13
Standard deviation     0.433086
Proportion of Variance 0.014580
Cumulative Proportion  1.000000


H2ODimReductionMetrics: pca

No model metrics available for PCA



In [14]:
pca_fit5
# 9 PC to achieve 80% variance explaination

Model Details:

H2ODimReductionModel: pca
Model ID:  PCA_model_R_1519665318376_38 
Importance of components: 
                            pc1      pc2      pc3      pc4      pc5      pc6
Standard deviation     1.947521 1.308992 1.263122 1.142624 1.106834 0.967275
Proportion of Variance 0.241992 0.109323 0.101795 0.083300 0.078163 0.059695
Cumulative Proportion  0.241992 0.351315 0.453110 0.536409 0.614572 0.674267
                            pc7      pc8      pc9     pc10     pc11     pc12
Standard deviation     0.871763 0.826412 0.800318 0.746853 0.679142 0.649597
Proportion of Variance 0.048488 0.043574 0.040866 0.035588 0.029428 0.026923
Cumulative Proportion  0.722755 0.766329 0.807195 0.842784 0.872212 0.899135
                           pc13     pc14     pc15     pc16     pc17     pc18
Standard deviation     0.619018 0.497375 0.451491 0.438133 0.402839 0.381038
Proportion of Variance 0.024448 0.015784 0.013006 0.012248 0.010354 0.009263
Cumulative Proportion  0.923583 0.939366 0.

#### Represent original dataset with principal components

In [15]:
features_pca1 <- h2o.predict(pca_fit1, predictors[, c(as.character(pca_features1))])
features_pca2 <- h2o.predict(pca_fit2, predictors[, c(as.character(pca_features2))])
#features_pca3 <- h2o.predict(pca_fit3, predictors[, c(as.character(pca_features3))])
# Housing variables are not chosen to be PCA transformed bacause 6 out of 8 dimensions are needed for 
# representing 80% of the variance.
features_pca4 <- h2o.predict(pca_fit4, predictors[, c(as.character(pca_features4))])
features_pca5 <- h2o.predict(pca_fit5, predictors[, c(as.character(pca_features5))])

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [16]:
names(features_pca1) = paste0("credit_", names(features_pca1))
names(features_pca2) = paste0("finance_", names(features_pca2))
names(features_pca4) = paste0("neighborhood_", names(features_pca4))
names(features_pca5) = paste0("trade_", names(features_pca5))

In [17]:
names(features_pca1)

[1] "credit_PC1"  "credit_PC2"  "credit_PC3"  "credit_PC4"  "credit_PC5" 
 [6] "credit_PC6"  "credit_PC7"  "credit_PC8"  "credit_PC9"  "credit_PC10"
[11] "credit_PC11" "credit_PC12" "credit_PC13" "credit_PC14" "credit_PC15"
[16] "credit_PC16" "credit_PC17" "credit_PC18"

In [18]:
full <- predictors[, c(as.character(lasso_features1$Variable))]
full = h2o.cbind(full, features_pca1[, 1:10])
full = h2o.cbind(full, features_pca2[, 1:4])
full = h2o.cbind(full, features_pca4[, 1:7])
full = h2o.cbind(full, features_pca5[, 1:9])

In [19]:
dim(full)

[1] 4732698     105

In [20]:
names(full)

[1] "score_C4"                         "iat51y5_C4"                      
  [3] "iat52_C4"                         "icc31_C4"                        
  [5] "i12ccpq2_C4"                      "i12ccpc1_C4"                     
  [7] "i12ccpq4_C4"                      "icc41_C4"                        
  [9] "i03inpq4_C4"                      "i12inpq4_C4"                     
 [11] "i12ccnb1_C4"                      "icc21_C4"                        
 [13] "iat89_C4"                         "icc33_C4"                        
 [15] "i12ccdu1_C4"                      "i12cctc1_C4"                     
 [17] "i12cctq1_C4"                      "i24txtb1_C4"                     
 [19] "i12cctl1_C4"                      "finscr_C4"                       
 [21] "iat31xam_C4"                      "imt31_C4"                        
 [23] "ire31_C4"                         "ifn102_C4"                       
 [25] "ihi20_C4"                         "imt20_C4"                        
 [27] "iat33_C4"                         "iat34_C4"                        
 [29] "iat34xam_C4"                      "HealthScore_C5"                  
 [31] "OriginalOwner_C1"                 "Housing.Units.Percent.5.Units_C3"
 [33] "curaddr_C4"                       "LotSize_C1"                      
 [35] "EstMarketValue_C1"                "Mortgage.Liability_C3"           
 [37] "Living.Area.Square.Feet_num"      "Length.of.Residence_num"         
 [39] "Target.Narrow.Band.Income_num"    "Number.of.Active.Sources_C3"     
 [41] "IssAgeALB"                        "CEN_tr_pctOwnOccValGE100K"       
 [43] "CEN_tr_pctOwnOccSecondMort"       "CEN_bg_pctSeasonalHousingUnits"  
 [45] "CEN_bg_pctHHincomeLT15K"          "CEN_tr_pctHHincomeGE200K"        
 [47] "CEN_tr_pctHHInvestIncome"         "CEN_tr_pctHHSocialSecurityIncome"
 [49] "CEN_bg_pctLiveAloneHH"            "CEN_bg_pctWorkforceForProfit"    
 [51] "CEN_bg_pctConstructionIndustry"   "CEN_bg_pctEducationIndustry"     
 [53] "CEN_tr_pctWorkforceGovt"          "CEN_tr_pctHSGrad"                
 [55] "iat01_C4"                         "iat02_C4"                        
 [57] "iat07_C4"                         "iau01y5_C4"                      
 [59] "ibr07_C4"                         "ibr41_C4"                        
 [61] "ifn01_C4"                         "ifn01y5_C4"                      
 [63] "ihi01y5_C4"                       "iin01y5_C4"                      
 [65] "imt01_C4"                         "imt01y5_C4"                      
 [67] "imt02y5_C4"                       "imt07_C4"                        
 [69] "imt41_C4"                         "imt96_C4"                        
 [71] "ire103_C4"                        "is004_C4"                        
 [73] "is004y5_C4"                       "ibr33_C4"                        
 [75] "ibr34_C4"                         "credit_PC1"                      
 [77] "credit_PC2"                       "credit_PC3"                      
 [79] "credit_PC4"                       "credit_PC5"                      
 [81] "credit_PC6"                       "credit_PC7"                      
 [83] "credit_PC8"                       "credit_PC9"                      
 [85] "credit_PC10"                      "finance_PC1"                     
 [87] "finance_PC2"                      "finance_PC3"                     
 [89] "finance_PC4"                      "neighborhood_PC1"                
 [91] "neighborhood_PC2"                 "neighborhood_PC3"                
 [93] "neighborhood_PC4"                 "neighborhood_PC5"                
 [95] "neighborhood_PC6"                 "neighborhood_PC7"                
 [97] "trade_PC1"                        "trade_PC2"                       
 [99] "trade_PC3"                        "trade_PC4"                       
[101] "trade_PC5"                        "trade_PC6"                       
[103] "trade_PC7"                        "trade_PC8"                       
[105] "trade_PC9"

In [21]:
pca_reduced_LASSO = full[, !(names(full) %in% c(as.character(pca_features1), as.character(pca_features2), 
                    as.character(pca_features4), as.character(pca_features5)))]

In [22]:
class(pca_reduced_LASSO)

[1] "H2OFrame"

In [23]:
h2o.exportFile(pca_reduced_LASSO, 'pca_reduced_LASSO.csv')

  |======================================================================| 100%


#### Train Kmeans model on PCA reduced LASSO selected features data set

In [74]:
pca_reduced_predictors <- h2o.importFile(path = normalizePath("/mnt/UW/outputDataset/pca_reduced_LASSO.csv")
                                         , destination_frame = "pca_reduced_predictors")

pca_reduced_predictors$C1 <- NULL

  |======================================================================| 100%


In [75]:
split <- h2o.splitFrame(pca_reduced_predictors, ratios = c(0.5, 0.2), seed = -1)
training = split[[1]]
validation = split[[2]]

In [76]:
dim(training)

[1] 2366306      44

In [78]:
x = names(training)

#### Optimal K

In [90]:
## For h2o kmeans, missing values are automatically imputed by the column mean of the training data
fit_kmeans <- function(training_frame, k, init){
    kmeans_fit <- h2o.kmeans(training_frame = training_frame, x, validation_frame = validation, nfolds = 8,
                             standardize = TRUE, seed = 1234, k = k, init = init)
    return(kmeans_fit)
}

In [101]:
var_explained_plusplus = c()
i=1

for (k in 3:13){
    model <- fit_kmeans(training, k, "PlusPlus")
    var_explained_plusplus[i] = (1-model@model$model_summary$within_cluster_sum_of_squares/model@model$model_summary$total_sum_of_squares)
    i = i +1
}

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [107]:
var_explained_plusplus

[1] 0.05976871 0.09632788 0.11561184 0.12779478 0.14661314 0.16606679
 [7] 0.17580157 0.19046981 0.20109095 0.20829528 0.21216428

In [108]:
### PlusPlus initialization, elbow curve
plot(c(3:13), var_explained_pluplus)

In [96]:
var_explained_random = c()
i=1

for (k in 3:13){
    model <- fit_kmeans(training, k, "Random")
    var_explained_random[i] = (1-model@model$model_summary$within_cluster_sum_of_squares/model@model$model_summary$total_sum_of_squares)
    i = i + 1
}

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [97]:
var_explained_random

[1] 0.07549735 0.09974153 0.11874860 0.13748630 0.15561795 0.16793094
 [7] 0.17443512 0.19233893 0.20349395 0.21251173 0.22161176

In [98]:
var_explained_furthest = c()
i=1

for (k in 3:13){
    model <- fit_kmeans(training, k, "Furthest")
    var_explained_furthest[i] = (1-model@model$model_summary$within_cluster_sum_of_squares/model@model$model_summary$total_sum_of_squares)
    i = i + 1
}
var_explained_furthest

  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


[1] 0.05428557 0.09243607 0.11148246 0.13539123 0.14614920 0.15773602
 [7] 0.16581279 0.17543484 0.19407499 0.20794732 0.20036704

In [109]:
# Save elbow curve data of three initialization methods
df_Kmeans_LASSO_Cleaned <- data.frame(k = c(3:13), 
                                      plusplus = c(var_explained_plusplus),
                                      random = c(var_explained_random), 
                                      furthest = c(var_explained_furthest))

In [111]:
saveRDS(df_Kmeans_LASSO_Cleaned, "Kmeans_LASSO_Cleaned.rds")

In [ ]:
ls()